# Lorentz matrix doodles

Idea: see what it takes to do things "just like a Lorentz matrix."

In [ ]:
%%capture
%matplotlib inline
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import math
import unittest

# To get equations the look like, well, equations, use the following.
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from IPython.display import display

# Tools for manipulating quaternions written by D. Sweetser.
from QH import QH;

from IPython.core.display import display, HTML, Math, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
t, x, y, z = sp.symbols("t x y z")

In [ ]:
v_txyz = sp.Matrix([t,x,y,z])

In [ ]:
Lyx = sp.Matrix([[1, 0, 0, 0],
                 [0, 0, -1, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1]])
display(Lyx)
display(Lyx * v_txyz)

Notice that each term is algebraically isolated from the others. To do this with quaternions, the same thing must be done - terms must be isolated from each other. This can be done using the three conjugates and a function to flip signs. For t, one uses the standard conjugates. To isolate x and y needs the action of the first and second conjugates. TO get z, all three conjugates and the sign flip.

In [ ]:
q_txyz = QH([t, x, y, z])

In [ ]:
q_txyz.add(q_txyz.conj()).print_state("2 t")
q_txyz.add(q_txyz.conj(1)).print_state("2 x")
q_txyz.add(q_txyz.conj(2)).print_state("2 y")
q_txyz.add(q_txyz.conj().conj(1).conj(2).flip_signs()).print_state("2 z")

Now twist the $x$ and $y$ states with a product with $k$.

In [ ]:
q_txyz.add(q_txyz.conj(1)).product(QH().q_k(-1)).print_state("2 x")
q_txyz.add(q_txyz.conj(2)).product(QH().q_k(-1)).print_state("2 y")

Just add it all up.

In [ ]:
q_txyz.add(q_txyz.conj()).add(q_txyz.add(q_txyz.conj(1)).product(QH().q_k(-1))).add(q_txyz.add(q_txyz.conj(2)).product(QH().q_k(-1))).add(q_txyz.add(q_txyz.conj().conj(1).conj(2).flip_signs())).product(QH().q_1(1/2)).display_q("tyxz")